# Validaciones Diarias Sistemas de Información UT

## CONFIGURACIONES Y CARGA DE ARCHIVOS


### IMPORTACIÓN DE LIBRERIAS


In [35]:
import pandas as pd
from datetime import datetime, timedelta
import pytz
import smtplib
from email.message import EmailMessage
import json

### CARGA DE ARCHIVOS

Este modulo permite la carga de los archivos en formato xlsx y csv según el caso e informa cuantas filas y columnas tiene el archivo cargado.

Archivos Necesarios


*   PQRS_Formato_Servicios (Umbrella)
*   PR_Formato (Umbrella) mes actual
*   GEN_Sitios_Activos_SI (Umbrella)
*   Lista de categorias (Umbrella)
*   LOG PQRSD (NetAdvisor) desde 01-06-2023 a fecha actual
*   Si_LOG_API_CCC (Umbrella)


In [ ]:
# Cargue PQRSD_Formato_Servicios
path ='/content/PQRSD_Formato_Servicios-2025-06-09.xlsx'
data = pd.read_excel(path)
data.shape

#Usar en caso de ejecutar localmente
#path2 = 'C:/Users/William Wallace/Desktop/PQRSD_Formato_Servicios-2024-07-24.xlsx'
#data_Categorias = pd.read_excel(path2)

(138065, 45)

In [ ]:
# Cargue paradas de reloj del mes - PR_Formato

path3 = '/content/PR_Formato-2025-06-09.xlsx'
data_paradas = pd.read_excel(path3)
data_paradas.shape

#Usar en caso de ejecutar localmente
#path3 = 'C:/Users/William Wallace/Desktop/PR_Formato-2024-07-24.xlsx'

(65498, 12)

In [ ]:
# Cargue lista personalizada - GEN_Sitios_Activos_SI
path4 = '/content/GEN_Sitios_Activos_SI-2025-06-09.xlsx'
data_id_punto = pd.read_excel(path4)
data_id_punto.shape

#Usar en caso de ejecutar localmente
#path4 = 'C:/Users/William Wallace/Desktop/GEN_Sitios_Activos_SI-2024-07-24.xlsx'

(6649, 46)

In [ ]:
# Cargue LOG PQRSD (NetAdvisor)
path5 = '/content/LOG 09062025.csv'
log_pqrs_datawifi = pd.read_csv(path5)
log_pqrs_datawifi.shape

#Usar en caso de ejecutar localmente
#path5 = 'C:/Users/William Wallace/Desktop/GEN_Sitios_Activos_SI-2024-07-24.xlsx'

(103529, 9)

In [ ]:
# Cargue lista categoria y subcategoria
path ='/content/Listas Personalizadas Listado Categorias PQRS_2025-06-09_08-27-26.xlsx'
categorias_subcategorias = pd.read_excel(path)
categorias_subcategorias.shape

(108, 52)

In [37]:
# Cargue log integracion CCC
path ='/content/SI_LOG_API_CCC-2025-06-09.xlsx'
integracion_ccc = pd.read_excel(path)
integracion_ccc.shape

(925, 8)

### CONFIGURACIONES DATAFRAMES

In [ ]:
# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_rows', None)

#dataframe reporte PQRSD_Formato_Servicios
dataframe = pd.DataFrame(data,columns=data.columns)
dataframe['Fecha_Cierre_PQRS'] = pd.to_datetime(dataframe['Fecha_Cierre_PQRS'])
dataframe['Hora_Cierre'] = dataframe['Fecha_Cierre_PQRS'].dt.hour.fillna(-1).astype(int)
dataframe['dia_fecha_Cierre'] = dataframe['Fecha_Cierre_PQRS'].dt.day.fillna(-1).astype(int)
dataframe['mes_fecha_Cierre'] = dataframe['Fecha_Cierre_PQRS'].dt.month.fillna(-1).astype(int)
dataframe['año_fecha_Cierre'] = dataframe['Fecha_Cierre_PQRS'].dt.year.fillna(-1).astype(int)

dataframe['Fecha_Creacion'] = pd.to_datetime(dataframe['Fecha_Creacion'])
dataframe['Hora_Creacion'] = dataframe['Fecha_Creacion'].dt.hour.fillna(-1).astype(int)
dataframe['dia_Fecha_Creacion'] = dataframe['Fecha_Creacion'].dt.day.fillna(-1).astype(int)
dataframe['mes_Fecha_Creacion'] = dataframe['Fecha_Creacion'].dt.month.fillna(-1).astype(int)
dataframe['año_Fecha_Creacion'] = dataframe['Fecha_Creacion'].dt.year.fillna(-1).astype(int)

dataframe['fecha_creacion_agenda'] = pd.to_datetime(dataframe['fecha_creacion_agenda'])

# Función para extraer horas de una cadena de tiempo
def extraer_horas(tiempo):
    partes = tiempo.split(':')
    horas = int(partes[0])
    return horas

#dataframe reporte paradas de reloj del mes
dataframe_Paradas = pd.DataFrame(data_paradas,columns=data_paradas.columns)
dataframe_Paradas['Hora_total_parada'] = dataframe_Paradas['tiempo_parada'].apply(extraer_horas)

dataframe_Paradas['fecha_ceracion_pqrs'] = pd.to_datetime(dataframe_Paradas['fecha_ceracion_pqrs'])
dataframe_Paradas['fecha_cierre_ticket'] = pd.to_datetime(dataframe_Paradas['fecha_cierre_ticket'])

dataframe_Paradas['dateparada'] = pd.to_datetime(dataframe_Paradas['dateparada'])
dataframe_Paradas['Dia_Parada'] = dataframe_Paradas['dateparada'].dt.day
dataframe_Paradas['dia_semana'] = dataframe_Paradas['dateparada'].dt.dayofweek
dataframe_Paradas['Hora_inicio'] = dataframe_Paradas['dateparada'].dt.hour
dataframe_Paradas['finishparada'] = pd.to_datetime(dataframe_Paradas['finishparada'])
dataframe_Paradas['Hora_fin'] = dataframe_Paradas['finishparada'].dt.hour

# Configuraciones de fechas
fecha_actual = datetime.now()
numero_dia = fecha_actual.day
mes_actual = fecha_actual.month
año_actual = fecha_actual.year
fecha = datetime.now().date()

dataframe_horarios = pd.DataFrame(data_id_punto,columns=data_id_punto.columns)

dataframe_log_pqrs_datawifi = pd.DataFrame(log_pqrs_datawifi,columns=log_pqrs_datawifi.columns)

print(fecha_actual)
print(numero_dia)
print(mes_actual)
print(año_actual)
print(fecha)

2025-06-09 13:35:05.856441
9
6
2025
2025-06-09


## VALIDACIONES


### Validación 1 - PQRSD SIN JOBID

Esta validación se encarga de verificar si existen mas de 119 PQRSD sin jobid.   
Retorna:
*    PQRSD sin jobid
*    Mensaje de error o exito.

In [ ]:
#validacion 1 PQRSD sin JOBID

data['Fecha_Creacion'] = pd.to_datetime(data['Fecha_Creacion'], errors='coerce')
nulos = data[(data['sv_job_id'].isna()) & (data['Fecha_Creacion'] > '2023-01-01')]
total = nulos.shape[0]

nulos = nulos[['ID','GRUPO','PRIORIDAD']]

if nulos.shape[0] > 0:
    nulos = nulos.to_html(index=False)
    #print ("Revisar, hay PQRSD agendados sin parada Fuera de Horario MDA")
    error1= f"Total: <b>{total}</b><br>{nulos}"
else:
    #print ("todos los pqrsd agendados tienen parada de reloj")
    error1="OK - Sin Novedades"

print(error1)

OK - Sin Novedades


### Validación 2 - PQRSD SIN FECHA MAXIMA DE ATENCIÓN
Esta validación verifica que todos los PQRSD agendados tengan fecha maxima de atención.  
Retorna:
*    La cantidad de PQRSD agendados con fecha máxima
*    La cantidad de PQRSD agendados **SIN** fecha máxima
*    y mensaje de error o exito.

In [ ]:
df_filtrado = dataframe[(dataframe['Estado'] == "Agendar") & (dataframe['fecha_maxima_atencion'].notnull())]
total_agendados=df_filtrado.shape[0]
print ("Cantidad de PQRSD agendados con fecha maxima:")
print (total_agendados)
df_filtrado2 = dataframe[(dataframe['Estado'] == "Agendar") & (dataframe['fecha_maxima_atencion'].isnull())]
total_agendados2=df_filtrado2.shape[0]
print ("Cantidad de PQRSD agendados SIN fecha maxima:")
print (total_agendados2)

df_filtrado2 = df_filtrado2[['ID','GRUPO','PRIORIDAD']]

if df_filtrado2.shape[0] > 0:
    df_filtrado2 = df_filtrado2.to_html(index=False)
    error2= f"Total: <b>{total_agendados2}</b> <br>{df_filtrado2}"
else:
    error2="OK - Sin Novedades"

print(error2)

Cantidad de PQRSD agendados con fecha maxima:
3417
Cantidad de PQRSD agendados SIN fecha maxima:
0
OK - Sin Novedades


### VALIDACIÓN 3 - PQRSD AGENDADOS SIN CORRECTIVO
Esta validación verifica que si el PQRSD esta en estado agendado tenga su respectivo Mantenimiento Correctivo.  
Retorna:
*    La cantidad de PQRSD agendados con correctivo.
*    La cantidad de PQRSD agendados **SIN** correctivo.
*    Mensaje de error o exito.

In [ ]:
#validacion3 PQRSD AGENDADO SIN CORRECTIVO

df_filtrado_validacion3 = dataframe[(dataframe['Estado'] == "Agendar") & (dataframe['CORRECTIVO'].notnull())]
#total_agendados=df_filtrado["Estado"].value_counts()
total_correctivos=df_filtrado_validacion3.shape[0]
print ("Cantidad de PQRSD agendados con correctivo:")
print (total_correctivos)
df_filtrado2_validacion3 = dataframe[(dataframe['Estado'] == "Agendar") & (dataframe['CORRECTIVO'].isnull())]
total_sin_correctivo=df_filtrado2_validacion3.shape[0]
print ("Cantidad de PQRSD agendados SIN correctivo:")
print (total_sin_correctivo)

df_filtrado2_validacion3 = df_filtrado2_validacion3[['ID','GRUPO','PRIORIDAD']]

if df_filtrado2_validacion3.shape[0] > 0:
    df_filtrado2_validacion3 = df_filtrado2_validacion3.to_html(index=False)
    error3= f"Total: <b>{total_sin_correctivo}</b> <br>{df_filtrado2_validacion3}"
else:
    error3="OK - Sin Novedades"

print(error3)

Cantidad de PQRSD agendados con correctivo:
3417
Cantidad de PQRSD agendados SIN correctivo:
0
OK - Sin Novedades


### VALIDACIÓN 4 - CIERRE DE PQRSD FUERA DE HORARIO

Esta validación verifica si existen PQRSD de las subcategorias **CD-FALLA EN EJECUCIÓN DE PRUEBA DE VELOCIDAD 5 DÍAS CALENDARIO** o **CD-MEDICIÓN DIRECTA DE VELOCIDAD EFECTIVA DE TRANSMISIÓN DE DATOS** cerrados despues de las 21:00 y antes de las 06:00 del mes en curso.  

Retorna:
*    La cantidad de PQRSD cerrados fuera de horario.
*    Si existen PQRSD cerrados fuera de horario retorna el **UUID** y **GRUPO**.
*    Mensaje de error o exito.

In [ ]:
#validacion 4 Fecha de cierre - verificar cierres fuera de horario
subcategorias = ['CD-FALLA EN EJECUCIÓN DE PRUEBA DE VELOCIDAD 5 DÍAS CALENDARIO', 'CD-MEDICIÓN DIRECTA DE VELOCIDAD EFECTIVA DE TRANSMISIÓN DE DATOS']

df_filtrado_antes_validacion4 = dataframe[(dataframe['Estado'] == "Cerrar") & (dataframe['CATEGORIA'] == "CD-CENTROS DE GESTIÓN SERVICIO") & (dataframe['SUBCATEGORIA'].isin(subcategorias)) & (dataframe['Hora_Cierre'] >= 21) & ( dataframe['mes_fecha_Cierre'] == mes_actual ) &  ( dataframe['año_fecha_Cierre'] == año_actual ) ]
df_filtrado_despues_validacion4 = dataframe[(dataframe['Estado'] == "Cerrar") & (dataframe['CATEGORIA'] == "CD-CENTROS DE GESTIÓN SERVICIO") & (dataframe['SUBCATEGORIA'].isin(subcategorias)) & (dataframe['Hora_Cierre'] < 6) & ( dataframe['mes_fecha_Cierre'] == mes_actual ) & ( dataframe['año_fecha_Cierre'] == año_actual ) ]

Fuera_de_horario_antes=df_filtrado_antes_validacion4.shape[0]
Fuera_de_horario_despues=df_filtrado_despues_validacion4.shape[0]
Fuera_de_horario_total=Fuera_de_horario_antes+Fuera_de_horario_despues

print ("Cantidad de PQRSD cerrados antes de las 6:")
print(Fuera_de_horario_antes)

print ("Cantidad de PQRSD cerrados despues de las 21:")
Fuera_de_horario_despues

print ("Cantidad de PQRSD cerrados fuera de horario:")
print (Fuera_de_horario_total)

df_filtrado_antes_validacion4 = df_filtrado_antes_validacion4[['ID','GRUPO','PRIORIDAD','Fecha_Creacion','Fecha_Cierre_PQRS']]

if Fuera_de_horario_total > 0:
    df_filtrado_antes_validacion4 = df_filtrado_antes_validacion4.to_html(index=False)
    error4= f"Total: <b>{Fuera_de_horario_antes}</b> <br>{df_filtrado_antes_validacion4}"
else:
    error4="OK - Sin Novedades"

print(error4)

Cantidad de PQRSD cerrados antes de las 6:
0
Cantidad de PQRSD cerrados despues de las 21:
Cantidad de PQRSD cerrados fuera de horario:
0
OK - Sin Novedades


### VALIDACIÓN 5 - PQRSD AGENDADOS SIN PARADA DE RELOJ FUERA DE HORARIO CD

Esta validación verifica que los PQRSD en estado **Agendar** tengan la parada de reloj Fuera de Horario CD.  

Retorna:
*    La cantidad de PQRSD sin parada de reloj Fuera de Horario CD.
*    Si existen PQRSD sin parada de reloj Fuera de Horario CD retorna el **UUID** y **GRUPO**.
*    Mensaje de error o exito.

In [ ]:
#validacion 5 verificar que todos los pqrsd agendados tienen parada de fuera de horario cd

df_filtrado_validacion5_agendados = dataframe[(dataframe['Estado'] == "Agendar")]
df_filtrado_validacion5_fuera_cd = dataframe_Paradas[(dataframe_Paradas['tipo_parada'] == "Fuera de Horario CD") & (dataframe_Paradas['Dia_Parada'] == numero_dia)]
resultvalidacion5 = pd.merge(df_filtrado_validacion5_agendados,df_filtrado_validacion5_fuera_cd, left_on='ID', right_on='uuid', how='left')
resultvalidacion5_2 = resultvalidacion5[(resultvalidacion5['uuid'].isnull())]
Sin_parada_cd=resultvalidacion5_2.shape[0]
print(resultvalidacion5_2.shape[0])
resultvalidacion5_2 = resultvalidacion5_2[['ID','GRUPO','PRIORIDAD','Fecha_Creacion','Fecha_Cierre_PQRS']]

if Sin_parada_cd > 0:
    resultvalidacion5_2 = resultvalidacion5_2.to_html(index=False)
    error5= f"Total: <b>{Sin_parada_cd}</b> <br>{resultvalidacion5_2}"
else:
    error5="OK - Sin Novedades"

print(error5)

0
OK - Sin Novedades


### VALIDACIÓN 6 - PQRSD ASIGNADOS A MESA DE AYUDA SIN PARADA DE RELOJ FUERA DE HORARIO MDA

Esta validación verifica que los PQRSD en estado **Asignado Mesa de Ayuda** tengan la parada de reloj Fuera de Horario MDA.  

Retorna:
*    La cantidad de PQRSD sin parada de reloj Fuera de Horario MDA.
*    Si existen PQRSD sin parada de reloj Fuera de Horario MDA retorna el **UUID** y **GRUPO**.
*    Mensaje de error o exito.

In [ ]:
#validacion 6 verificar que todos los pqrsd en mesa de ayuda tienen parada de fuera de horario MDA

df_filtrado_validacion6_agendados = dataframe[(dataframe['Estado'] == "Asignado Mesa de Ayuda")]
df_filtrado_validacion6_fuera_mda = dataframe_Paradas[(dataframe_Paradas['tipo_parada'] == "Fuera de Horario MDA") & (dataframe_Paradas['Dia_Parada'] == numero_dia)]
resultvalidacion6 = pd.merge(df_filtrado_validacion6_agendados,df_filtrado_validacion6_fuera_mda, left_on='ID', right_on='uuid', how='left')
resultvalidacion6_2 = resultvalidacion6[(resultvalidacion6['uuid'].isnull())]

Sin_parada_mda=resultvalidacion6_2.shape[0]

resultvalidacion6_2 = resultvalidacion6_2[['ID','GRUPO','PRIORIDAD','Fecha_Creacion','Fecha_Cierre_PQRS']]

if Sin_parada_mda > 0:
    resultvalidacion6_2 = resultvalidacion6_2.to_html(index=False)
    error6= f"Total: <b>{Sin_parada_mda}</b> <br>{resultvalidacion6_2}"
else:
    error6="OK - Sin Novedades"

print(error6)

Total: <b>1</b> <br><table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>ID</th>
      <th>GRUPO</th>
      <th>PRIORIDAD</th>
      <th>Fecha_Creacion</th>
      <th>Fecha_Cierre_PQRS</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>6C8E83-5272B7</td>
      <td>GRUPO 3A</td>
      <td>2</td>
      <td>2024-03-04 07:02:53</td>
      <td>NaT</td>
    </tr>
  </tbody>
</table>


### VALIDACIÓN 7 - PARADAS DE RELOJ MAYOR A 24 HORAS

Esta validación verifica que **NO** existan paradas de reloj Fuera de Horario MDA o Fuera de Horario CD mayores a 24 Horas.  

Retorna:
*    La cantidad de Paradas de reloj mayores a 23:59 Horas.
*    Mensaje de error o exito.



In [ ]:
#validacion 7 paradas con tiempo mayor a 24 horas
dataframe_Paradas['Hora_total_parada']

df_filtrado_validacion7 = dataframe_Paradas[(dataframe_Paradas['Hora_total_parada'] > 24) & (dataframe_Paradas['tipo_parada'] != "Evento excepcional") & (dataframe_Paradas['tipo_parada'] != "Diagnostico 8 Dias") ]
exede23horas=df_filtrado_validacion7.shape[0]

df_filtrado_validacion7 = df_filtrado_validacion7[['id_parada','idbeneficiario','uuid','fecha_ceracion_pqrs','dateparada','finishparada','fecha_cierre_ticket']]

if exede23horas > 0:
    df_filtrado_validacion7 = df_filtrado_validacion7.to_html(index=False)
    error7= f"Total: <b>{exede23horas}</b> {df_filtrado_validacion7}"
else:
    error7="OK - Sin Novedades"

print(error7)

OK - Sin Novedades


### VALIDACIÓN 8 - PARADAS DE RELOJ FUERA DE HORARIO CD - CUANDO EL CD NO TIENE HORARIOS  
Esta validación verifica los horarios de los Centros Digitales y realiza la validación con las paradas de reloj Fuera de Horario CD  

Retorna:  
*    Cantidad de paradas Fuera de horario CD dia por dia mal puestas.  
*    Mensaje de error o exito.

In [ ]:
#validacion 8 parada fuera de horario cd -fuera del horario cargado
#dataframe_horarios
#dataframe_Paradas

dataframe_horarios['Consolidado_Lunes'] = dataframe_horarios['Consolidado_Lunes'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Martes'] = dataframe_horarios['Consolidado_Martes'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Miercoles'] = dataframe_horarios['Consolidado_Miercoles'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Jueves'] = dataframe_horarios['Consolidado_Jueves'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Viernes'] = dataframe_horarios['Consolidado_Viernes'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Sabado'] = dataframe_horarios['Consolidado_Sabado'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Domingo'] = dataframe_horarios['Consolidado_Domingo'].fillna('00:00:00 a 00:00:00').astype(str)

def extraer_horas(rango_tiempo):
    # Dividir el rango de tiempo en dos partes
    try:
        tiempo_inicio, tiempo_fin = rango_tiempo.split(' a ')

    except ValueError as e:
        print(f"Error al convertir horas: {e}")
        return None, None
    return tiempo_inicio, tiempo_fin

# Aplicar la función a la columna 'tiempo' y crear nuevas columnas 'hora_inicio' y 'hora_fin'
dataframe_horarios[['Lunes_hora_inicio', 'Lunes_hora_fin']] = dataframe_horarios['Consolidado_Lunes'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Martes_hora_inicio', 'Martes_hora_fin']] = dataframe_horarios['Consolidado_Martes'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Miercoles_hora_inicio', 'Miercoles_hora_fin']] = dataframe_horarios['Consolidado_Miercoles'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Jueves_hora_inicio', 'Jueves_hora_fin']] = dataframe_horarios['Consolidado_Jueves'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Viernes_hora_inicio', 'Viernes_hora_fin']] = dataframe_horarios['Consolidado_Viernes'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Sabado_hora_inicio', 'Sabado_hora_fin']] = dataframe_horarios['Consolidado_Sabado'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Domingo_hora_inicio', 'Domingo_hora_fin']] = dataframe_horarios['Consolidado_Domingo'].apply(lambda x: pd.Series(extraer_horas(x)))

df_filtrado8_paradas_fuera_cd = dataframe_Paradas[(dataframe_Paradas['tipo_parada'] == "Fuera de Horario CD")]

resultvalidacion81 = pd.merge(df_filtrado8_paradas_fuera_cd,dataframe_horarios, left_on='idbeneficiario', right_on='Id_Beneficiario', how='left')
resultvalidacion8 = pd.merge(resultvalidacion81,dataframe, left_on='uuid', right_on='ID', how='inner')

resultvalidacion8['Hora_inicio'] = pd.to_numeric(resultvalidacion8['Hora_inicio'], errors='coerce')

resultvalidacion8['Lunes_hora_fin'] = pd.to_numeric(resultvalidacion8['Lunes_hora_fin'], errors='coerce')
resultvalidacion8['Lunes_hora_inicio'] = pd.to_numeric(resultvalidacion8['Lunes_hora_inicio'], errors='coerce')
resultvalidacion8['Martes_hora_fin'] = pd.to_numeric(resultvalidacion8['Martes_hora_fin'], errors='coerce')
resultvalidacion8['Martes_hora_inicio'] = pd.to_numeric(resultvalidacion8['Martes_hora_inicio'], errors='coerce')
resultvalidacion8['Miercoles_hora_fin'] = pd.to_numeric(resultvalidacion8['Miercoles_hora_fin'], errors='coerce')
resultvalidacion8['Miercoles_hora_inicio'] = pd.to_numeric(resultvalidacion8['Miercoles_hora_inicio'], errors='coerce')
resultvalidacion8['Jueves_hora_fin'] = pd.to_numeric(resultvalidacion8['Jueves_hora_fin'], errors='coerce')
resultvalidacion8['Jueves_hora_inicio'] = pd.to_numeric(resultvalidacion8['Jueves_hora_inicio'], errors='coerce')
resultvalidacion8['Viernes_hora_fin'] = pd.to_numeric(resultvalidacion8['Viernes_hora_fin'], errors='coerce')
resultvalidacion8['Viernes_hora_inicio'] = pd.to_numeric(resultvalidacion8['Viernes_hora_inicio'], errors='coerce')
resultvalidacion8['Sabado_hora_fin'] = pd.to_numeric(resultvalidacion8['Sabado_hora_fin'], errors='coerce')
resultvalidacion8['Sabado_hora_inicio'] = pd.to_numeric(resultvalidacion8['Sabado_hora_inicio'], errors='coerce')
resultvalidacion8['Domingo_hora_fin'] = pd.to_numeric(resultvalidacion8['Domingo_hora_fin'], errors='coerce')
resultvalidacion8['Domingo_hora_inicio'] = pd.to_numeric(resultvalidacion8['Domingo_hora_inicio'], errors='coerce')


#resultvalidacion8['Lunes_hora_fin'] = pd.to_datetime(resultvalidacion8['Lunes_hora_fin'], format='%H:%M:%S').dt.time
#resultvalidacion8['Lunes_hora_inicio'] = pd.to_datetime(resultvalidacion8['Lunes_hora_inicio'], errors='coerce').dt.time


resultvalidacion8['dia_semana'] = pd.to_numeric(resultvalidacion8['dia_semana'], errors='coerce')

df_filtrado_validacion8_lunes = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Lunes_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Lunes_hora_inicio']) & (resultvalidacion8['dia_semana'] == 0)]
falla_lunes=df_filtrado_validacion8_lunes.shape[0]
df_filtrado_validacion8_martes = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Martes_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Martes_hora_inicio']) & (resultvalidacion8['dia_semana'] == 1)]
falla_martes=df_filtrado_validacion8_martes.shape[0]
df_filtrado_validacion8_miercoles = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Miercoles_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Miercoles_hora_inicio']) & (resultvalidacion8['dia_semana'] == 2)]
falla_miercoles=df_filtrado_validacion8_miercoles.shape[0]
df_filtrado_validacion8_jueves = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Jueves_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Jueves_hora_inicio']) & (resultvalidacion8['dia_semana'] == 3)]
falla_jueves=df_filtrado_validacion8_jueves.shape[0]
df_filtrado_validacion8_viernes = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Viernes_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Viernes_hora_inicio']) & (resultvalidacion8['dia_semana'] == 4)]
falla_viernes=df_filtrado_validacion8_viernes.shape[0]
df_filtrado_validacion8_sabado = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Sabado_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Sabado_hora_inicio']) & (resultvalidacion8['dia_semana'] == 5)]
falla_sabado=df_filtrado_validacion8_sabado.shape[0]
df_filtrado_validacion8_domingo = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Domingo_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Domingo_hora_inicio']) & (resultvalidacion8['dia_semana'] == 6)]
falla_domingo=df_filtrado_validacion8_domingo.shape[0]

print (falla_lunes)
print (falla_martes)
print (falla_miercoles)
print (falla_jueves)
print (falla_viernes)
print (falla_sabado)
print (falla_domingo)

#revisa los dias sin horario

df_filtrado_validacion8_lunes_sin_horario = resultvalidacion8[( resultvalidacion8['Lunes_hora_inicio']==0) & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 0)]
falla_lunes_sin_horario=df_filtrado_validacion8_lunes_sin_horario.shape[0]
df_filtrado_validacion8_martes_sin_horario = resultvalidacion8[( resultvalidacion8['Martes_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 1)]
falla_martes_sin_horario=df_filtrado_validacion8_martes_sin_horario.shape[0]
df_filtrado_validacion8_miercoles_sin_horario = resultvalidacion8[( resultvalidacion8['Miercoles_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 2)]
falla_miercoles_sin_horario=df_filtrado_validacion8_miercoles_sin_horario.shape[0]
df_filtrado_validacion8_jueves_sin_horario = resultvalidacion8[( resultvalidacion8['Jueves_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 3)]
falla_jueves_sin_horario=df_filtrado_validacion8_jueves_sin_horario.shape[0]
df_filtrado_validacion8_viernes_sin_horario = resultvalidacion8[( resultvalidacion8['Viernes_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 4)]
falla_viernes_sin_horario=df_filtrado_validacion8_viernes_sin_horario.shape[0]
df_filtrado_validacion8_sabado_sin_horario = resultvalidacion8[( resultvalidacion8['Sabado_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 5)]
falla_sabado_sin_horario=df_filtrado_validacion8_sabado_sin_horario.shape[0]
df_filtrado_validacion8_domingo_sin_horario = resultvalidacion8[( resultvalidacion8['Domingo_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 6)]
falla_domingo_sin_horario=df_filtrado_validacion8_domingo_sin_horario.shape[0]

print ("sin horario")
print (falla_lunes_sin_horario)
print (falla_martes_sin_horario)
print (falla_miercoles_sin_horario)
print (falla_jueves_sin_horario)
print (falla_viernes_sin_horario)
print (falla_sabado_sin_horario)
print (falla_domingo_sin_horario)

# Guardar las predicciones en un archivo Excel
#df_filtrado8_paradas_fuera_cd.to_excel('C:/Users/William Wallace/Desktop/dias-horario.xlsx', index=False)
#df_filtrado8_paradas_fuera_cd.to_excel('C:/Users/William Wallace/Desktop/cerrado-fuera-horario2.xlsx', index=False)

fallas_total=falla_lunes+falla_martes+falla_miercoles+falla_jueves+falla_viernes+falla_sabado+falla_domingo
fallas_total_sin_horario=falla_lunes_sin_horario+falla_martes_sin_horario+falla_miercoles_sin_horario+falla_jueves_sin_horario+falla_viernes_sin_horario+falla_sabado_sin_horario+falla_domingo_sin_horario

if fallas_total > 0:
    error8=(f"Total: {fallas_total}")
else:
    error8="OK - Sin Novedades"

if fallas_total_sin_horario > 0:
    print ("Revisar, hay paradas de reloj como si tuviera horario pero no tiene cargado")
    print (fallas_total_sin_horario)
    error8_1="Revisar, hay paradas de reloj como si tuviera horario pero no tiene cargado"
else:
    print ("Las paradas de reloj fuera de horario CD, son correctas")
    error8_1="OK - Sin Novedades"

0
0
0
0
0
0
0
sin horario
0
0
0
0
0
0
0
Las paradas de reloj fuera de horario CD, son correctas


### VALIDACIÓN 9 - CATEGORIA - SUBCATEGORIA - CODIGO CATEGORIA - PRIORIDAD  
Esta validación verifica que la categoria, subcategoria, codigo de categoria y prioridad si corresponda a la suministrada por la lista categorias en el arbol de tipificación  

Retorna:  
*    Cantidad de PQRSD categoria, subcategoria, codigo de categoria y prioridad diferente.  
*    Mensaje de error o exito.

In [ ]:
# Reemplazar NaN con 0
categorias_subcategorias['Column 5'] = categorias_subcategorias['Column 5'].fillna(0)
categorias_subcategorias['Column 5'] = categorias_subcategorias['Column 5'].astype(int)
categorias_subcategorias_concatenado = pd.DataFrame()
categorias_subcategorias_concatenado['Concatenado'] = categorias_subcategorias['Column 1'].astype(str) + categorias_subcategorias['Column 2'].astype(str) + categorias_subcategorias['Column 4'].astype(str) + categorias_subcategorias['Column 5'].astype(str)

# Convertir la columna 'Concatenado' a mayúsculas
categorias_subcategorias_concatenado['Concatenado'] = categorias_subcategorias_concatenado['Concatenado'].str.upper()


id_cate_subca = data[['ID','GRUPO','Estado']].copy()
id_cate_subca['Concatenar'] = data[['CATEGORIA', 'SUBCATEGORIA', 'Código_Categoría', 'PRIORIDAD']].apply(lambda x: ''.join(x.astype(str)), axis=1)

id_cate_subca['Concatenar'] = id_cate_subca['Concatenar'].str.upper()

id_cate_subca['Validación'] = id_cate_subca['Concatenar'].isin(categorias_subcategorias_concatenado['Concatenado'])


error_cate_subca = id_cate_subca[~id_cate_subca['Validación']]

error_cate_subca = error_cate_subca[['ID','GRUPO','Estado']]

if(error_cate_subca.shape[0] > 0):
    error9 = f"Total: <b>{error_cate_subca.shape[0]}</b> <br> {error_cate_subca}"
else:
    error9 = "OK - Sin Novedades"

print(error9)

OK - Sin Novedades


### VALIDACIÓN 10 Y 11 - FECHA DE CIERRE NO SEA MAYOR NI MENOR A 1 MINUTO
Esta validación verifica que la fecha de cierre del PQRSD no sea mayor ni menor a 1 minuto

Retorna:  
*    Cantidad de PQRSD con error.  
*    Mensaje de error o exito.

In [ ]:
# Validación 10 - Verificar fecha de cierre

# Filtrar el DataFrame por el estado 'Cerrar'
df_filtrado_validacion10_PQRSD_cerrados = dataframe[(dataframe['Estado'] == "Cerrar")]

# Realizar la unión entre los DataFrames utilizando 'ID' y 'uuid' como claves
resultvalidacion10 = pd.merge(df_filtrado_validacion10_PQRSD_cerrados, dataframe_Paradas, left_on='ID', right_on='uuid', how='inner')

# Rellenar los valores NaN con False
resultvalidacion10 = resultvalidacion10.fillna(False)

# Seleccionar las columnas deseadas
columnas_deseadas = ['id_parada', 'uuid', 'idbeneficiario', 'idpqr', 'fecha_ceracion_pqrs', 'Fecha_Cierre_PQRS', 'fecha_cierre_ticket']
resultvalidacion10 = resultvalidacion10[columnas_deseadas]
#print (resultvalidacion10)

# Asegurarse de que las columnas de fecha estén en formato datetime
resultvalidacion10['Fecha_Cierre_PQRS'] = pd.to_datetime(resultvalidacion10['Fecha_Cierre_PQRS'], errors='coerce')
resultvalidacion10['fecha_cierre_ticket'] = pd.to_datetime(resultvalidacion10['fecha_cierre_ticket'], errors='coerce')

# Calcular la diferencia en minutos entre las dos fechas
resultvalidacion10['diferencia_minutos'] = ((resultvalidacion10['Fecha_Cierre_PQRS'] - resultvalidacion10['fecha_cierre_ticket']).dt.total_seconds() / 60)
#print (resultvalidacion10)

# Reemplazar los valores NaN con 0 antes de la conversión a entero

df_filtrado_10_1 = resultvalidacion10[resultvalidacion10['diferencia_minutos'].isnull()]


resultvalidacion10['diferencia_minutos'] = resultvalidacion10['diferencia_minutos'].fillna(0)

# Convertir la diferencia en minutos a entero
resultvalidacion10['diferencia_minutos'] = resultvalidacion10['diferencia_minutos'].astype(int)

# Filtrar las filas donde la diferencia en minutos sea menor que -1 o mayor que 1
df_filtrado_10 = resultvalidacion10[(resultvalidacion10['diferencia_minutos'] < -1) | (resultvalidacion10['diferencia_minutos'] > 1)]

# Contar el número de registros con diferencias
diferencia_cierre = df_filtrado_10.shape[0]

# Contar el número de registros con paradas abiertas y pqrsd cerrado
diferencia_cierre_10_1 = df_filtrado_10_1.shape[0]

#print (diferencia_cierre_10_1)


df_filtrado_10 = df_filtrado_10[['id_parada','uuid','idpqr','diferencia_minutos']]

df_filtrado_10_1 = df_filtrado_10_1[['id_parada','uuid','idpqr','diferencia_minutos']]

# Condicional para imprimir el resultado de la validación
if diferencia_cierre > 0:
    df_filtrado_10 = df_filtrado_10.to_html(index=False)
    error10 = f"Total: {diferencia_cierre} <br> {df_filtrado_10}"
else:
    error10 = "OK - Sin Novedades"

# Condicional para imprimir el resultado de la validación 10.1
if diferencia_cierre_10_1 > 0:
    df_filtrado_10_1 = df_filtrado_10_1.to_html(index=False)
    error11 = f"Total: {diferencia_cierre_10_1} <br> {df_filtrado_10_1}"
else:
    error11 = "OK - Sin Novedades"

print(error10)
print(error11)

OK - Sin Novedades
OK - Sin Novedades


### VALIDACIÓN 12 - PQRSD ABIERTOS PARA SITIOS SALIENTES
Esta validación verifica que los sitios salientes no tengan pqrsd Abiertas

Retorna:  
*    Cantidad de PQRSD con error.  
*    Mensaje de error o exito.

In [ ]:
validacion_12 = data.copy()
validacion_12 = validacion_12[validacion_12['GRUPO'].str.contains('saliente', case=False, na=False) & (validacion_12['Estado'] != 'Cerrar')]

total_fallas_validacion_12 = validacion_12.shape[0]

validacion_12 = validacion_12[['ID','ID_Beneficiario', 'GRUPO', 'Estado']]

# Verificar si el dataframe filtrado está vacío
if total_fallas_validacion_12 > 0:
    validacion_12 = validacion_12.to_html(index=False)
    error12 = (f"Total: <b>{total_fallas_validacion_12}</b> <br>{validacion_12}")
else:
    error12 = "OK - Sin Novedades"

print(error12)

OK - Sin Novedades


### VALIDACIÓN 13 - PQRS ENVIADAS DATAWIFI VS CREADAS UMBRELLA
Esta validación verifica que la cantdad de PQRS enviadas desde datawifi sea igual a las PQRSD creadas

Retorna:
*    Cantidad de PQRSD con error.
*    Mensaje de error o exito.

In [ ]:
data['Fecha_Creacion'] = pd.to_datetime(data['Fecha_Creacion'])
# Convertir la columna 'Fecha_Creacion' al formato 'YYYY-MM-DD'
data['Fecha_Creacion'] = data['Fecha_Creacion'].dt.strftime('%Y-%m-%d')

log_pqrs_datawifi['FECHA DE CREACIÓN'] = pd.to_datetime(log_pqrs_datawifi['FECHA DE CREACIÓN'])
# Convertir la columna 'Fecha_Creacion' al formato 'YYYY-MM-DD'
log_pqrs_datawifi['FECHA DE CREACIÓN'] = log_pqrs_datawifi['FECHA DE CREACIÓN'].dt.strftime('%Y-%m-%d')

fecha_validacion = fecha.strftime('%Y-%m-%d')

pqrs_datawifi_hoy = log_pqrs_datawifi[(log_pqrs_datawifi['FECHA DE CREACIÓN'] == fecha_validacion)]
pqrs_umbrella_hoy = data[(data['Fecha_Creacion'] == fecha_validacion) & (data['Usuario_Creador'] == 'DATAWIFI.API')]
conteo_pqrs_umbrella_hoy = pqrs_umbrella_hoy.shape[0]
conteo_datawifi = pqrs_datawifi_hoy.shape[0]

if(conteo_pqrs_umbrella_hoy != conteo_datawifi):
    error13 = f"Diferencia: <b>{(conteo_datawifi - conteo_pqrs_umbrella_hoy)}</b> <br> Enviadas DataWifi: <b>{conteo_datawifi}</b> <br> Creadas Umbrella: <b> {conteo_pqrs_umbrella_hoy}</b>"
else:
    error13 = f'OK, Sin Novedades <br> Enviadas DataWifi: <b>{conteo_datawifi}</b> <br> Creadas Umbrella: <b> {conteo_pqrs_umbrella_hoy}</b>'

print(error13)

OK, Sin Novedades <br> Enviadas DataWifi: <b>166</b> <br> Creadas Umbrella: <b> 166</b>


### VALIDACIÓN 14, 15, 16 - ALARMAS GENERADAS DATAWIFI POR SUBCATEGORIA
Esta validación verifica la cantidad de alarmas generadas por datawifi discriminadas por subcategoria

Retorna:
*    Cantidad de alarmas con error.
*    Mensaje de error o exito.

In [ ]:

log_pqrs_datawifi['FECHA DE CREACIÓN'] = pd.to_datetime(log_pqrs_datawifi['FECHA DE CREACIÓN'])
# Convertir la columna 'Fecha_Creacion' al formato 'YYYY-MM-DD'
log_pqrs_datawifi['FECHA DE CREACIÓN'] = log_pqrs_datawifi['FECHA DE CREACIÓN'].dt.strftime('%Y-%m-%d')

fecha_validacion = fecha.strftime('%Y-%m-%d')

pqrs_datawifi_hoy_velocidad = log_pqrs_datawifi[
    (log_pqrs_datawifi['FECHA DE CREACIÓN'] == fecha_validacion) &
    (
        (log_pqrs_datawifi['SUBCATEGORYS'] == 'Registro de medición de velocidad de transmisión de datos') |
        (log_pqrs_datawifi['SUBCATEGORYS'] == 'NO registro de medición de velocidad')
    )
]

if(pqrs_datawifi_hoy_velocidad.shape[0] <= 0):
    error14 = f"No se generarón alarmas por concepto de velocidad en DataWifi"
else:
    error14 = f'Velocidad: <b>{pqrs_datawifi_hoy_velocidad.shape[0]}</b>'


pqrs_datawifi_hoy_outdoor = log_pqrs_datawifi[
    (log_pqrs_datawifi['FECHA DE CREACIÓN'] == fecha_validacion) &
    (
        (log_pqrs_datawifi['SUBCATEGORYS'] == 'NO registro de trafico de la Zona exterior') |
        (log_pqrs_datawifi['SUBCATEGORYS'] == 'Registro de trafico de la Zona exterior')
    )
]

if(pqrs_datawifi_hoy_outdoor.shape[0] <= 0):
    error15 = f"No se generarón alarmas por concepto de Trafico Outdoor en DataWifi"
else:
    error15 = f'Trafico Outdoor: <b>{pqrs_datawifi_hoy_outdoor.shape[0]}</b>'


pqrs_datawifi_hoy_indoor = log_pqrs_datawifi[
    (log_pqrs_datawifi['FECHA DE CREACIÓN'] == fecha_validacion) &
    (
        (log_pqrs_datawifi['SUBCATEGORYS'] == 'NO registro de trafico de la Zona interior') |
        (log_pqrs_datawifi['SUBCATEGORYS'] == 'Registro de trafico de la Zona interior')
    )
]

if(pqrs_datawifi_hoy_indoor.shape[0] <= 0):
    error16 = f"No se generarón alarmas por concepto de Trafico Indoor en DataWifi"
else:
    error16 = f'Trafico Indoor: <b>{pqrs_datawifi_hoy_indoor.shape[0]}</b>'

print(error14)
print(error15)
print(error16)

Velocidad: <b>41</b>
Trafico Outdoor: <b>62</b>
Trafico Indoor: <b>63</b>


### VALIDACIÓN 17 - HORAS EN LAS QUE NO SE CERRARON PQRSD POR CONCEPTO DE VELOCIDAD
Esta validación verifica Y las horas en las que no se cerraron PQRSD por concepto de velocidad.

Retorna:
*    Cantidad de alarmas con error.
*    Mensaje de error o exito.

In [ ]:
logs_velocidad = log_pqrs_datawifi.copy()

# Asegurarse de que 'FECHA DE CIERRE' esté en formato datetime
logs_velocidad['FECHA DE CIERRE'] = pd.to_datetime(logs_velocidad['FECHA DE CIERRE'], errors='coerce')

# Obtener la fecha de ayer
ayer = (datetime.now() - timedelta(days=1)).date()

# Filtrar solo los registros cerrados el día de ayer
logs_ayer = logs_velocidad[logs_velocidad['FECHA DE CIERRE'].dt.date == ayer]

logs_ayer = logs_ayer.copy()
logs_ayer['HORA'] = logs_ayer['FECHA DE CIERRE'].dt.hour

logs_ayer.loc[:, 'HORA'] = logs_ayer['FECHA DE CIERRE'].dt.hour

logs_filtrado = logs_ayer[logs_ayer['SUBCATEGORYS'] == 'Registro de medición de velocidad de transmisión de datos'].copy()
logs_filtrado['HORA'] = logs_filtrado['FECHA DE CIERRE'].dt.hour

# Agrupar por hora y contar
resumen = logs_filtrado.groupby('HORA').size().reset_index(name='CANTIDAD CERRADOS MEDICIÓN VELOCIDAD')

# Asegurar que aparezcan todas las horas del 0 al 23 (aunque no haya datos)
todas_las_horas = pd.DataFrame({'HORA': range(24)})
resultado_final_datawifi = todas_las_horas.merge(resumen, on='HORA', how='left').fillna(0)
resultado_final_datawifi['CANTIDAD CERRADOS MEDICIÓN VELOCIDAD'] = resultado_final_datawifi['CANTIDAD CERRADOS MEDICIÓN VELOCIDAD'].astype(int)

# Filtrar las horas entre 6 y 21 con cantidad igual a 0
horas_sin_cierres = resultado_final_datawifi[
    (resultado_final_datawifi['CANTIDAD CERRADOS MEDICIÓN VELOCIDAD'] == 0) &
    (resultado_final_datawifi['HORA'] >= 6) &
    (resultado_final_datawifi['HORA'] <= 21)
]

if horas_sin_cierres.shape[0] > 0:
    horas_sin_cierres = horas_sin_cierres.to_html(index=False)
    error17 = horas_sin_cierres
else:
    error17 = "OK - Sin Novedades"

print(error17)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>HORA</th>
      <th>CANTIDAD CERRADOS MEDICIÓN VELOCIDAD</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <td>7</td>
      <td>0</td>
    </tr>
    <tr>
      <td>8</td>
      <td>0</td>
    </tr>
    <tr>
      <td>9</td>
      <td>0</td>
    </tr>
    <tr>
      <td>10</td>
      <td>0</td>
    </tr>
    <tr>
      <td>13</td>
      <td>0</td>
    </tr>
    <tr>
      <td>14</td>
      <td>0</td>
    </tr>
    <tr>
      <td>15</td>
      <td>0</td>
    </tr>
    <tr>
      <td>16</td>
      <td>0</td>
    </tr>
    <tr>
      <td>17</td>
      <td>0</td>
    </tr>
    <tr>
      <td>18</td>
      <td>0</td>
    </tr>
    <tr>
      <td>19</td>
      <td>0</td>
    </tr>
    <tr>
      <td>20</td>
      <td>0</td>
    </tr>
    <tr>
      <td>21</td>
      <td>0</td>
    </tr>
  </tbody>
</table>


### VALIDACIÓN 18 - HORAS EN LAS QUE NO SE CERRARON PQRSD POR CONCEPTO DE TRAFICO INDOOR
Esta validación verifica Y las horas en las que no se cerraron PQRSD por concepto de velocidad.

Retorna:
*    Cantidad de alarmas con error.
*    Mensaje de error o exito.

In [ ]:
logs_indoor = log_pqrs_datawifi.copy()

# Asegurarse de que 'FECHA DE CIERRE' esté en formato datetime
logs_indoor['FECHA DE CIERRE'] = pd.to_datetime(logs_indoor['FECHA DE CIERRE'], errors='coerce')

# Obtener la fecha de ayer
ayer = (datetime.now() - timedelta(days=1)).date()

# Filtrar solo los registros cerrados el día de ayer
logs_ayer = logs_indoor[logs_indoor['FECHA DE CIERRE'].dt.date == ayer]

logs_ayer = logs_ayer.copy()
logs_ayer['HORA'] = logs_ayer['FECHA DE CIERRE'].dt.hour

logs_ayer.loc[:, 'HORA'] = logs_ayer['FECHA DE CIERRE'].dt.hour

logs_filtrado = logs_ayer[logs_ayer['SUBCATEGORYS'] == 'Registro de trafico de la Zona interior'].copy()
logs_filtrado['HORA'] = logs_filtrado['FECHA DE CIERRE'].dt.hour

# Agrupar por hora y contar
resumen = logs_filtrado.groupby('HORA').size().reset_index(name='CANTIDAD CERRADOS INDOOR')

# Asegurar que aparezcan todas las horas del 0 al 23 (aunque no haya datos)
todas_las_horas = pd.DataFrame({'HORA': range(24)})
resultado_final = todas_las_horas.merge(resumen, on='HORA', how='left').fillna(0)
resultado_final['CANTIDAD CERRADOS INDOOR'] = resultado_final['CANTIDAD CERRADOS INDOOR'].astype(int)

# Filtrar las horas entre 6 y 21 con cantidad igual a 0
horas_sin_cierres_indoor = resultado_final[
    (resultado_final['CANTIDAD CERRADOS INDOOR'] == 0) #&
    # (resultado_final['hora'] >= 6) &
    # (resultado_final['hora'] <= 21)
]

if horas_sin_cierres_indoor.shape[0] > 0:
    horas_sin_cierres_indoor = horas_sin_cierres_indoor.to_html(index=False)
    error18 = horas_sin_cierres_indoor
else:
    error18 = "OK - Sin Novedades"

print(error18)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>HORA</th>
      <th>CANTIDAD CERRADOS INDOOR</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td>2</td>
      <td>0</td>
    </tr>
    <tr>
      <td>3</td>
      <td>0</td>
    </tr>
    <tr>
      <td>4</td>
      <td>0</td>
    </tr>
    <tr>
      <td>5</td>
      <td>0</td>
    </tr>
    <tr>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <td>8</td>
      <td>0</td>
    </tr>
    <tr>
      <td>16</td>
      <td>0</td>
    </tr>
    <tr>
      <td>22</td>
      <td>0</td>
    </tr>
    <tr>
      <td>23</td>
      <td>0</td>
    </tr>
  </tbody>
</table>


### VALIDACIÓN 19 - HORAS EN LAS QUE NO SE CERRARON PQRSD POR CONCEPTO DE TRAFICO OUTDOOR
Esta validación verifica Y las horas en las que no se cerraron PQRSD por concepto de velocidad.

Retorna:
*    Cantidad de alarmas con error.
*    Mensaje de error o exito.

In [ ]:
logs_outdoor = log_pqrs_datawifi.copy()

# Asegurarse de que 'FECHA DE CIERRE' esté en formato datetime
logs_outdoor['FECHA DE CIERRE'] = pd.to_datetime(logs_outdoor['FECHA DE CIERRE'], errors='coerce')

# Obtener la fecha de ayer
ayer = (datetime.now() - timedelta(days=1)).date()

# Filtrar solo los registros cerrados el día de ayer
logs_ayer = logs_outdoor[logs_outdoor['FECHA DE CIERRE'].dt.date == ayer]

logs_ayer = logs_ayer.copy()
logs_ayer['HORA'] = logs_ayer['FECHA DE CIERRE'].dt.hour

logs_ayer.loc[:, 'HORA'] = logs_ayer['FECHA DE CIERRE'].dt.hour

logs_filtrado = logs_ayer[logs_ayer['SUBCATEGORYS'] == 'Registro de trafico de la Zona exterior'].copy()
logs_filtrado['HORA'] = logs_filtrado['FECHA DE CIERRE'].dt.hour

# Agrupar por hora y contar
resumen = logs_filtrado.groupby('HORA').size().reset_index(name='CANTIDAD CERRADOS OUTDOOR')

# Asegurar que aparezcan todas las horas del 0 al 23 (aunque no haya datos)
todas_las_horas = pd.DataFrame({'HORA': range(24)})
resultado_final = todas_las_horas.merge(resumen, on='HORA', how='left').fillna(0)
resultado_final['CANTIDAD CERRADOS OUTDOOR'] = resultado_final['CANTIDAD CERRADOS OUTDOOR'].astype(int)

# Filtrar las horas entre 6 y 21 con cantidad igual a 0
horas_sin_cierres_outdoor = resultado_final[
    (resultado_final['CANTIDAD CERRADOS OUTDOOR'] == 0) #&
    # (resultado_final['hora'] >= 6) &
    # (resultado_final['hora'] <= 21)
]

if horas_sin_cierres_outdoor.shape[0] > 0:
    horas_sin_cierres_outdoor = horas_sin_cierres_outdoor.to_html(index=False)
    error19 = horas_sin_cierres_outdoor
else:
    error19 = "OK - Sin Novedades"

print(error19)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>HORA</th>
      <th>CANTIDAD CERRADOS OUTDOOR</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td>2</td>
      <td>0</td>
    </tr>
    <tr>
      <td>3</td>
      <td>0</td>
    </tr>
    <tr>
      <td>4</td>
      <td>0</td>
    </tr>
    <tr>
      <td>5</td>
      <td>0</td>
    </tr>
    <tr>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <td>8</td>
      <td>0</td>
    </tr>
    <tr>
      <td>16</td>
      <td>0</td>
    </tr>
    <tr>
      <td>22</td>
      <td>0</td>
    </tr>
    <tr>
      <td>23</td>
      <td>0</td>
    </tr>
  </tbody>
</table>


### VALIDACIÓN 20 - PQRS ENVIADA DATAWIFI SIN UUID
Esta validación verifica que las peticiones enviadas desde datawifi hayan retornado el UUID

Retorna:
*    Cantidad de PQRSD con error.
*    Mensaje de error o exito.

In [ ]:
# Filtrar filas donde la columna PQR está vacía
vacios = pqrs_datawifi_hoy[pqrs_datawifi_hoy['PQR'].isna()]

# Crear nuevo DataFrame con las columnas requeridas
vacios = vacios[['IDENTIFICADOR BENEFICIARIO', 'PQR', 'SUBCATEGORYS','STATUS']]
total = vacios.shape[0]

# Verificar si hay filas con errores
if total > 0:
    vacios = vacios.to_html(index=False)
    error20 = (f"Total: <b>{total}</b> <br>{vacios}")
else:
    error20 = "OK - Sin Novedades"

print(error20)

OK - Sin Novedades


### VALIDACIÓN 21 - ESTADO UMBRELLA vs ESTADO DATAWIFI
Esta validación verifica que el estado del PQRSD en umbrella coincida con el registrado en Datawifi

Retorna:
*    Cantidad de PQRSD con error.
*    Mensaje de error o exito.

In [ ]:
#Verificar estados umbrella vs datawifi
log_pqrs_datawifi_estados = log_pqrs_datawifi.copy()
pqrs_estados_umbrella = data.copy()
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Cerrar','Cerrado')
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Agendar','Abierto')
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Asignado Mesa de Ayuda','Abierto')

merged_df = pd.merge(pqrs_estados_umbrella, log_pqrs_datawifi_estados, how='inner', left_on='ID', right_on='PQR')

merged_df = merged_df[merged_df['Estado'] != merged_df['STATUS']]

merged_df = merged_df.rename(columns={'Estado': 'Estado_Umbrella', 'Fecha_Cierre_PQRS':'Fecha_Cierre_PQRS_Umbrella', 'STATUS':'Estado_Datawifi', 'FECHA DE CIERRE':'Fecha_Cierre_PQRS_Datawifi','PQR':'PQRS'})

merged_df = merged_df[['ID_Beneficiario', 'GRUPO','ID', 'Estado_Umbrella', 'Estado_Datawifi','Fecha_Cierre_PQRS_Umbrella', 'Fecha_Cierre_PQRS_Datawifi']]

total = merged_df.shape[0]

if total > 0:
    merged_df = merged_df.to_html(index=False)
    error21 = (f"Total: <b>{total}</b> <br>{merged_df}")
else:
    error21 = "OK - Sin Novedades"

print(error21)

OK - Sin Novedades


### VALIDACIÓN 22 - ESTADO DATAWIFI vs ESTADO UMBRELLA
Esta validación verifica que el estado del PQRSD en umbrella coincida con el registrado en Datawifi

Retorna:
*    Cantidad de PQRSD con error.
*    Mensaje de error o exito.

In [ ]:
#Verificar estados umbrella vs datawifi
log_pqrs_datawifi_estados = log_pqrs_datawifi.copy()
pqrs_estados_umbrella = data.copy()
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Cerrar','Cerrado')
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Agendar','Abierto')
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Asignado Mesa de Ayuda','Abierto')

merged_df = pd.merge(log_pqrs_datawifi_estados, pqrs_estados_umbrella, how='inner', left_on='PQR', right_on='ID')

merged_df = merged_df[merged_df['Estado'] != merged_df['STATUS']]

merged_df = merged_df.rename(columns={'Estado': 'Estado_Umbrella', 'Fecha_Cierre_PQRS':'Fecha_Cierre_PQRS_Umbrella', 'STATUS':'Estado_Datawifi', 'FECHA DE CIERRE':'Fecha_Cierre_PQRS_Datawifi','PQR':'PQRS'})

merged_df = merged_df[['ID_Beneficiario', 'GRUPO','ID', 'Estado_Umbrella', 'Estado_Datawifi','Fecha_Cierre_PQRS_Umbrella', 'Fecha_Cierre_PQRS_Datawifi']]

total = merged_df.shape[0]

if total > 0:
    merged_df = merged_df.to_html(index=False)
    error22 = (f"Total: <b>{total}</b> <br>{merged_df}")
else:
   error22 = "OK - Sin Novedades"

print(error22)


OK - Sin Novedades


 ### VALIDACIÓN 23 Y 24 PARADAS DE RELOJ ENERGÍA ALTERNATIVA DE PQRSD ABIERTAS Y CERRADAS

Esta validación verifica las paradas de reloj EnergÍa Alternativa de PQRSD abiertas y cerradas

Retorna:
*    Cantidad de PQRSD con error.
*    Mensaje de error o exito.

In [ ]:
dataframe_horarios_15=dataframe_horarios[['Id_Beneficiario','Tipo_Energia']]

dataframe_15=dataframe[['ID_Beneficiario','Fecha_Creacion','ID','año_fecha_Cierre','mes_fecha_Cierre','dia_fecha_Cierre','Hora_Cierre','Fecha_Cierre_PQRS','Estado']]
dataframe_15 = dataframe_15[dataframe_15['Fecha_Creacion'] > "2024-09-17 00:00:00"]

# Obtener el primer día del mes en curso
primer_dia_mes = datetime.today().replace(day=1)
# Formatear la fecha en el formato 'año-mes-día'
primer_dia_mes = primer_dia_mes.strftime('%Y-%m-%d')
primer_dia_mes_formateado = primer_dia_mes + " 09:09:09"
print (primer_dia_mes)
print (primer_dia_mes_formateado)

puntos_solares = dataframe_horarios_15[(dataframe_horarios_15['Tipo_Energia']=="SOLUCION SOLAR")]
puntos_solares.reset_index(drop=True, inplace=True)
pqrsd_solares=pd.merge(dataframe_15,puntos_solares, left_on='ID_Beneficiario', right_on='Id_Beneficiario', how='inner')

pqrsd_solares.loc[pqrsd_solares['Fecha_Creacion'] < primer_dia_mes, 'Fecha_Creacion'] = primer_dia_mes_formateado
pqrsd_solares['Hora_Creacion'] = pqrsd_solares['Fecha_Creacion'].dt.hour.fillna(-1).astype(int)
pqrsd_solares['dia_Fecha_Creacion'] = pqrsd_solares['Fecha_Creacion'].dt.day.fillna(-1).astype(int)
pqrsd_solares['mes_Fecha_Creacion'] = pqrsd_solares['Fecha_Creacion'].dt.month.fillna(-1).astype(int)
pqrsd_solares['año_Fecha_Creacion'] = pqrsd_solares['Fecha_Creacion'].dt.year.fillna(-1).astype(int)
pqrsd_solares.reset_index(drop=True, inplace=True)

#print (pqrsd_solares)

df_filtrado_validacion15_abiertos = pqrsd_solares[~(pqrsd_solares['Estado'] == "Cerrar")]
df_filtrado_validacion15_cerrados_mes = pqrsd_solares[(pqrsd_solares['Estado'] == "Cerrar") & (pqrsd_solares['año_fecha_Cierre'] == año_actual) & (pqrsd_solares['mes_fecha_Cierre'] == mes_actual)]
paradas_solares = dataframe_Paradas[(dataframe_Paradas['tipo_parada'] == "Energía Alternativa")]

df_filtrado_validacion15_abiertos.reset_index(drop=True, inplace=True)
df_filtrado_validacion15_cerrados_mes.reset_index(drop=True, inplace=True)

#print (df_filtrado_validacion15_abiertos)

#print (df_filtrado_validacion15_cerrados_mes)

cruce_abiertos = pd.merge(df_filtrado_validacion15_abiertos,paradas_solares, left_on='ID', right_on='uuid', how='left')
cruce_cerrados = pd.merge(df_filtrado_validacion15_cerrados_mes,paradas_solares, left_on='ID', right_on='uuid', how='left')
#print (cruce_cerrados)
cruce_abiertos.reset_index(drop=True, inplace=True)
cruce_cerrados.reset_index(drop=True, inplace=True)

#print(cruce_cerrados.columns)
cruce_abiertos['Conteo'] = cruce_abiertos.groupby(['ID', 'ID_Beneficiario', 'Tipo_Energia', 'Estado', 'Fecha_Creacion', 'dia_Fecha_Creacion', 'uuid'])['ID'].transform('count').fillna(0).astype(int)
cruce_cerrados['Conteo'] = cruce_cerrados.groupby(['ID', 'ID_Beneficiario', 'Tipo_Energia',  'Fecha_Creacion', 'dia_Fecha_Creacion','Estado','dia_fecha_Cierre','Fecha_Cierre_PQRS', 'uuid'])['ID'].transform('count').fillna(0).astype(int)

cruce_abiertos=cruce_abiertos[['ID', 'ID_Beneficiario', 'Tipo_Energia', 'Estado', 'Fecha_Creacion', 'dia_Fecha_Creacion','Hora_Creacion', 'uuid','Conteo']]
cruce_cerrados=cruce_cerrados[['ID', 'ID_Beneficiario', 'Tipo_Energia',  'Fecha_Creacion', 'dia_Fecha_Creacion','Hora_Creacion','Estado','dia_fecha_Cierre','Hora_Cierre','Fecha_Cierre_PQRS', 'uuid','Conteo']]

cruce_abiertos.loc[:, 'paradas_calculadas']=(numero_dia-cruce_abiertos['dia_Fecha_Creacion'])
cruce_abiertos.loc[cruce_abiertos['Hora_Creacion'] < 7, 'paradas_calculadas'] = (cruce_abiertos['paradas_calculadas']+1)
cruce_abiertos.loc[:, 'revision']=(cruce_abiertos['paradas_calculadas']-cruce_abiertos['Conteo'])
cruce_cerrados.loc[:, 'paradas_calculadas']=(cruce_cerrados['dia_fecha_Cierre']-cruce_cerrados['dia_Fecha_Creacion'])
cruce_cerrados.loc[cruce_cerrados['Hora_Creacion'] < 7, 'paradas_calculadas'] = (cruce_cerrados['paradas_calculadas']+1)
cruce_cerrados.loc[cruce_cerrados['Hora_Cierre'] > 18, 'paradas_calculadas'] = (cruce_cerrados['paradas_calculadas']+1)

cruce_cerrados['revision']=(cruce_cerrados['paradas_calculadas']-cruce_cerrados['Conteo'])

#print(cruce_abiertos[['Fecha_Creacion',  'Conteo', 'pardas_calculadas', 'revision']])
#print(cruce_cerrados[['Fecha_Creacion',  'Conteo', 'pardas_calculadas', 'revision','Hora_Cierre']])

fallas_abiertos = cruce_abiertos[cruce_abiertos['revision'] !=0 ]
fallas_cerrados = cruce_cerrados[cruce_cerrados['revision'] !=0 ]

fallas_abiertos = fallas_abiertos.groupby(['ID', 'ID_Beneficiario', 'Tipo_Energia', 'Estado', 'Fecha_Creacion', 'dia_Fecha_Creacion', 'uuid']).last().reset_index()
fallas_cerrados = fallas_cerrados.groupby(['ID', 'ID_Beneficiario', 'Tipo_Energia', 'Estado', 'Fecha_Creacion', 'dia_Fecha_Creacion', 'uuid']).last().reset_index()



total_fallas_abiertos = fallas_abiertos.shape[0]
total_fallas_cerrados = fallas_cerrados.shape[0]

if total_fallas_abiertos > 0:
    fallas_abiertos = fallas_abiertos.to_html(index=False)
    error23_abiertos=(f"Total: <b>{total_fallas_abiertos}</b> <br>{fallas_abiertos}")
else:
    error23_abiertos = "OK - Sin Novedades"

if total_fallas_cerrados > 0:
    fallas_cerrados = fallas_cerrados.to_html(index=False)
    error24_cerrados = (f"Total: <b>{total_fallas_cerrados}</b> <br>{fallas_cerrados}")
else:
    error24_cerrados = "OK - Sin Novedades"

print (error23_abiertos)
print (error24_cerrados)

2025-06-01
2025-06-01 09:09:09
OK - Sin Novedades
OK - Sin Novedades


### VALIDACIÓN 25 y 26 INTEGRACION PQRSD LAVENIR Y CCC

Esta validacion verifica si se presento alguna falla en la creacion de PQRSD ya sea por etiquetas o por algun otro inconveniente

Retorna:

Cantidad de PQRSD con error.
Mensaje de error o exito.

In [41]:

dataframe_falla_etiquetas=integracion_ccc[(integracion_ccc ['status']=="NO")].reset_index()

dataframe_falla_duplas=integracion_ccc.reset_index()

dataframe_falla_duplas['json'] = dataframe_falla_duplas['log'].apply(json.loads)
dataframe_falla_duplas['ref_id'] = dataframe_falla_duplas['json'].apply(lambda x: x.get('ref_id'))

conteo = dataframe_falla_duplas.groupby('ref_id').size().reset_index(name='conteo')
dataframe_falla_duplas = dataframe_falla_duplas.merge(conteo, on='ref_id', how='left')

print(dataframe_falla_duplas[['ref_id', 'conteo']])



             ref_id  conteo
0    RPRUEBA-000248     1.0
1              None     NaN
2          P-000999     1.0
3              None     NaN
4          P-000998     1.0
5              None     NaN
6          P-000997     5.0
7          P-000997     5.0
8          R-001467     1.0
9              None     NaN
10         R-001543     1.0
11             None     NaN
12         R-001541     1.0
13             None     NaN
14         R-001554     1.0
15             None     NaN
16         R-001560     1.0
17             None     NaN
18         R-001574     1.0
19             None     NaN
20         R-001590     1.0
21             None     NaN
22         R-001591     1.0
23             None     NaN
24         R-001592     1.0
25             None     NaN
26         R-001604     1.0
27             None     NaN
28         R-001605     1.0
29             None     NaN
30         R-001607     1.0
31             None     NaN
32         R-001608     1.0
33             None     NaN
34         R-001609 

## CONSOLIDADO ERRORES


In [ ]:
print ("VALIDACIÓN 1 PQRSD SIN JOBID")
print (error1)
print ("VALIDACIÓN 2 PQRSD SIN FECHA MAXIMA DE ATENCIÓN")
print (error2)
print ("VALIDACIÓN 3 PQRSD AGENDADO SIN CORRECTIVO")
print (error3)
print ("VALIDACIÓN 4 CIERRE DE PQRSD FUERA DE HORARIO")
print (error4)
print ("VALIDACIÓN 5 PQRSD AGENDADOS SIN PARADA DE RELOJ FUERA DE HORARIO CD")
print (error5)
print ("VALIDACIÓN 6 PQRSD ASIGNADOS A MESA DE AYUDA SIN PARADA DE RELOJ FUERA DE HORARIO MDA")
print (error6)
print ("VALIDACIÓN 7 PARADA DE RELOJ MAYOR A 24 HORAS")
print (error7)
print ("VALIDACIÓN 8 PARADAS DE RELOJ FUERA DE HORARIO CD - CUANDO EL CD NO TIENE HORARIOS")
print (error8)
print ("VALIDACIÓN 8.1 PARADAS DE RELOJ CREADAS COMO SI TUVIERA HORARIO PERO ESE DIA NO TIENE HORARIO RELACIONADO")
print (error8_1)
print ("VALIDACIÓN 9 CATEGORIA - SUBCATEGORIA - CODIGO CATEGORIA - PRIORIDAD")
print (error9)
print ("VALIDACIÓN 10 FECHA DE CIERRE NO SEA MAYOR NI MENOR A 1 MINUTO")
print (error10)
print ("VALIDACIÓN 11 FECHA DE CIERRE PARADA RELOJ VS PQRSD ESTADOS DIFERENTES")
print (error11)
print ("VALIDACIÓN 12 PQRSD ABIERTOS PARA SITIOS SALIENTES")
print (error12)
print ("VALIDACIÓN 13 PQRS ENVIADAS DATAWIFI VS CREADAS UMBRELLA")
print (error13)
print ("VALIDACIÓN 14, 15, 16 ALARMAS GENERADAS DATAWIFI POR SUBCATEGORIA")
print (error14)
print (error15)
print (error16)
print ("VALIDACIÓN 17 HORAS EN LAS QUE NO SE CERRARON PQRSD POR CONCEPTO DE VELOCIDAD")
print (error17)
print ("VALIDACIÓN 18 HORAS EN LAS QUE NO SE CERRARON PQRSD POR CONCEPTO DE TRAFICO INDOOR")
print (error18)
print ("VALIDACIÓN 19 HORAS EN LAS QUE NO SE CERRARON PQRSD POR CONCEPTO DE TRAFICO OUTDOOR")
print (error19)
print ("VALIDACIÓN 20 PQRS ENVIADA DATAWIFI SIN UUID")
print (error20)
print ("VALIDACIÓN 21 ESTADO UMBRELLA vs ESTADO DATAWIFI")
print (error21)
print ("VALIDACIÓN 22 ESTADO DATAWIFI vs ESTADO UMBRELLA")
print (error22)
print ("VALIDACIÓN 23 PARADAS DE RELOJ ENERGÍA ALTERNATIVA - PQRSD ABIERTOS")
print (error23_abiertos)
print ("VALIDACIÓN 24 PARADAS DE RELOJ ENERGÍA ALTERNATIVA - PQRSD CERRADOS")
print (error24_cerrados)

VALIDACIÓN 1 PQRSD SIN JOBID
OK - Sin Novedades
VALIDACIÓN 2 PQRSD SIN FECHA MAXIMA DE ATENCIÓN
OK - Sin Novedades
VALIDACIÓN 3 PQRSD AGENDADO SIN CORRECTIVO
OK - Sin Novedades
VALIDACIÓN 4 CIERRE DE PQRSD FUERA DE HORARIO
OK - Sin Novedades
VALIDACIÓN 5 PQRSD AGENDADOS SIN PARADA DE RELOJ FUERA DE HORARIO CD
OK - Sin Novedades
VALIDACIÓN 6 PQRSD ASIGNADOS A MESA DE AYUDA SIN PARADA DE RELOJ FUERA DE HORARIO MDA
Total: <b>1</b> <br><table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>ID</th>
      <th>GRUPO</th>
      <th>PRIORIDAD</th>
      <th>Fecha_Creacion</th>
      <th>Fecha_Cierre_PQRS</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>6C8E83-5272B7</td>
      <td>GRUPO 3A</td>
      <td>2</td>
      <td>2024-03-04 07:02:53</td>
      <td>NaT</td>
    </tr>
  </tbody>
</table>
VALIDACIÓN 7 PARADA DE RELOJ MAYOR A 24 HORAS
OK - Sin Novedades
VALIDACIÓN 8 PARADAS DE RELOJ FUERA DE HORARIO CD - CUANDO EL CD NO TIENE HORARIOS
OK - Sin Nov

## ENVIO CORREO

In [ ]:
nombres = ['Victor Campos', 'Liseth Guzmán', 'William Mora']

while True:
    for i, nombre in enumerate(nombres, 1):
        print(f"{i}. {nombre}")

    opcion = int(input("Seleccione una opcion: "))

    if 1 <= opcion <= len(nombres):
        nombre = nombres[opcion - 1]
        print(f"Has seleccionado: {nombre}")
        break  # Salir del bucle cuando se selecciona una opción válida
    else:
        print("Opción no válida, por favor intente de nuevo.\n")

# Variable de fecha

hora = datetime.now(pytz.timezone('America/Bogota')).strftime("%H:%M:%S")

# Configuración del servidor SMTP
email_subject = f"Revisiones diarias {fecha}"
sender_email_address = "wsistemasdeinformacion@gmail.com"
receiver_email_address = ["sistemasdeinformacionUT@sky.net.co", "nelson.miranda@sky.net.co"]

email_smtp = "smtp.gmail.com"
email_password = "ufwc spdj dcxv eopw"

# Creación del objeto EmailMessage
message = EmailMessage()

# Configuracion de las cabeceras del email
message['Subject'] = email_subject
message['From'] = sender_email_address
message['To'] = receiver_email_address

# Creación del cuerpo del correo
body = f"""
<html>
<head>
    <style>
        body {{
            font-family: 'Arial', sans-serif;
            font-size: 16px;
            color: #2c2c2c;
            background-color: #ffffff;
            padding: 20px;
        }}
        h1 {{
            color: #00264d;
            font-size: 30px;
            border-bottom: 2px solid #003366;
            padding-bottom: 5px;
        }}
        h3 {{
            color: #003366;
            font-size: 18px;
            margin-top: 25px;
            margin-bottom: 5px;
        }}
        p {{
            font-size: 16px;
            color: #444444;
            margin: 4px 0;
        }}
        b {{
            color: #000000;
        }}
        .highlight {{
            font-weight: bold;
            color: #003366;
        }}
        table {{
            width: auto;
            border-collapse: collapse;
            margin-top: 15px;
            font-size: 16px;
            margin-left: 0;
        }}
        th {{
            background-color: #003366;
            color: #ffffff;
            padding: 8px 12px;
            border: 1px solid #cccccc;
            text-align: center;
        }}
        td {{
            padding: 8px 12px;
            border: 1px solid #cccccc;
            text-align: center;
        }}
        tr:nth-child(even) {{
            background-color: #f4f7fb;
        }}
    </style>
</head>
<body>
    <h1>VALIDACIONES DIARIAS {fecha}</h1>

    <h3>VALIDACIÓN 1</h3>
    <p><b>PQRSD SIN JOBID:</b> {error1}</p>

    <h3>VALIDACIÓN 2</h3>
    <p><b>PQRSD SIN FECHA MÁXIMA DE ATENCIÓN:</b> {error2}</p>

    <h3>VALIDACIÓN 3</h3>
    <p><b>PQRSD AGENDADO SIN CORRECTIVO:</b> {error3}</p>

    <h3>VALIDACIÓN 4</h3>
    <p><b>CIERRES DE PQRSD FUERA DE HORARIO:</b> {error4}</p>

    <h3>VALIDACIÓN 5</h3>
    <p><b>PQRSD AGENDADOS SIN PARADA DE RELOJ FUERA DE HORARIO CD:</b> {error5}</p>

    <h3>VALIDACIÓN 6</h3>
    <p><b>PQRSD ASIGNADOS A MESA DE AYUDA SIN PARADA DE RELOJ FUERA DE HORARIO MDA:</b> {error6}</p>

    <h3>VALIDACIÓN 7</h3>
    <p><b>PARADAS DE RELOJ MAYOR A 24H:</b> {error7}</p>

    <h3>VALIDACIÓN 8</h3>
    <p><b>PARADAS DE RELOJ FUERA DE HORARIO CD - CUANDO EL CD NO TIENE HORARIOS:</b> {error8}</p>

    <h3>VALIDACIÓN 9</h3>
    <p><b>PARADAS DE RELOJ CREADAS COMO SI TUVIERA HORARIO PERO ESE DIA NO TIENE HORARIO RELACIONADO:</b> {error8_1}</p>

    <h3>VALIDACIÓN 10</h3>
    <p><b>CATEGORÍA - SUBCATEGORÍA - CÓDIGO CATEGORÍA - PRIORIDAD:</b> {error9}</p>

    <h3>VALIDACIÓN 11</h3>
    <p><b>FECHA DE CIERRE NO SEA MAYOR NI MENOR A 1 MINUTO:</b> {error10}</p>

    <h3>VALIDACIÓN 12</h3>
    <p><b>FECHA DE CIERRE PARADA RELOJ VS PQRSD ESTADOS DIFERENTES:</b> {error11}</p>

    <h3>VALIDACIÓN 13</h3>
    <p><b>PQRSD ABIERTOS PARA SITIOS SALIENTES:</b> {error12}</p>

    <h3>VALIDACIÓN 14</h3>
    <p><b>PQRS ENVIADAS DATAWIFI VS CREADAS UMBRELLA:</b> {error13}</p>


    <h3>VALIDACIÓN 15, 16 Y 17</h3>
    <p><b>ALARMAS GENERADAS DATAWIFI POR SUBCATEGORIA:</b>
        <p>{error14}</p>
        <p>{error15}</p>
        <p>{error16}</p>
    </p>

    <h3>VALIDACIÓN 18</h3>
    <p><b>HORAS EN LAS QUE NO SE CERRARON PQRSD POR CONCEPTO DE VELOCIDAD:</b> {error17}</p>

    <h3>VALIDACIÓN 19</h3>
    <p><b>HORAS EN LAS QUE NO SE CERRARON PQRSD POR CONCEPTO DE TRAFICO INDOOR:</b> {error18}</p>

    <h3>VALIDACIÓN 20</h3>
    <p><b>HORAS EN LAS QUE NO SE CERRARON PQRSD POR CONCEPTO DE TRAFICO OUTDOOR:</b> {error19}</p>

    <h3>VALIDACIÓN 21</h3>
    <p><b>PQRS ENVIADA DATAWIFI SIN UUID:</b> {error20}</p>

    <h3>VALIDACIÓN 22</h3>
    <p><b>ESTADO UMBRELLA vs ESTADO DATAWIFI:</b> {error21}</p>

    <h3>VALIDACIÓN 23</h3>
    <p><b>ESTADO DATAWIFI vs ESTADO UMBRELLA:</b> {error22}</p>

    <h3>VALIDACIÓN 24</h3>
    <p><b>PARADAS DE RELOJ ENERGÍA ALTERNATIVA - PQRSD ABIERTOS:</b> {error23_abiertos}</p>

    <h3>VALIDACIÓN 25</h3>
    <p><b>PARADAS DE RELOJ ENERGÍA ALTERNATIVA - PQRSD CERRADOS:</b> {error24_cerrados}</p>

    <br><br>
    <p style="font-size: 16px; color: #666;">Proceso automatizado ejecutado por: <b>{nombre}</b> a las {hora}</p>
</body>
</html>
"""



# Agregar el contenido HTML al mensaje
message.add_alternative(body, subtype='html')

# Variable server guarda configuracion del protocolo y puerto
server = smtplib.SMTP(email_smtp, '587')

# Identificación del cliente en el servidor SMTP
server.ehlo()

# Añade seguridad a la conexión
server.starttls()

# Inicio de sesion con las credenciales
server.login(sender_email_address, email_password)

# Envio correo
server.send_message(message)

# Cerrar conexión con el servidor
server.quit()

1. Victor Campos
2. Liseth Guzmán
3. William Mora
Seleccione una opcion: 1
Has seleccionado: Victor Campos


(221,
 b'2.0.0 closing connection d2e1a72fcca58-7482b0c8a6bsm5903873b3a.142 - gsmtp')